# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "../output_data/cities.csv"
city_data_df = pd.read_csv(city_data)
city_data_df.head()

,City,Latitude,Longitude,Humidity,Max Temperature,Cloudiness,Wind Speed,Country
0,rabaul,-4.1967,152.1721,78,80.71,68,3.67,PG
1,tasiilaq,65.6145,-37.6368,61,6.80,20,3.44,GL
2,dan khun thot,15.2085,101.7714,99,71.01,23,3.67,TH
3,puerto ayora,-0.7393,-90.3518,83,78.80,75,9.71,EC
4,meulaboh,4.1363,96.1285,89,76.66,74,4.47,ID


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Find max humidity for heatmap intensity 
city_data_df["Humidity"].max()

100

In [5]:
# Create a map using state centroid coordinates to set markers
locations = city_data_df[['Latitude', 'Longitude']]
humidity = city_data_df["Humidity"]


# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
perfect_weather = pd.DataFrame(city_data_df.loc[(city_data_df["Max Temperature"] >= 80) \
                                   & (city_data_df["Max Temperature"] <= 90)
                                   & (city_data_df["Humidity"] <= 60)
                                   & (city_data_df["Cloudiness"] <= 3)])


In [7]:
perfect_weather

,City,Latitude,Longitude,Humidity,Max Temperature,Cloudiness,Wind Speed,Country
56,panaba,21.2833,-88.2667,44,87.96,0,10.85,MX
98,coihaique,-45.5752,-72.0662,25,89.60,0,8.05,CL
171,ancud,-41.8697,-73.8203,44,82.99,0,6.91,CL
238,general roca,-39.0333,-67.5833,23,86.00,0,14.97,AR
252,vallenar,-28.5708,-70.7581,29,83.86,0,9.98,CL
323,bocana de paiwas,12.7857,-85.1227,55,83.88,3,9.66,NI
407,rawson,-43.3002,-65.1023,27,82.44,0,29.89,AR
448,bolgatanga,10.7856,-0.8514,13,81.25,0,4.45,GH
525,ovalle,-30.5983,-71.2003,39,80.94,0,9.84,CL


In [8]:
perfect_weather.dropna(how='any', inplace=True)

In [9]:
perfect_weather

,City,Latitude,Longitude,Humidity,Max Temperature,Cloudiness,Wind Speed,Country
56,panaba,21.2833,-88.2667,44,87.96,0,10.85,MX
98,coihaique,-45.5752,-72.0662,25,89.60,0,8.05,CL
171,ancud,-41.8697,-73.8203,44,82.99,0,6.91,CL
238,general roca,-39.0333,-67.5833,23,86.00,0,14.97,AR
252,vallenar,-28.5708,-70.7581,29,83.86,0,9.98,CL
323,bocana de paiwas,12.7857,-85.1227,55,83.88,3,9.66,NI
407,rawson,-43.3002,-65.1023,27,82.44,0,29.89,AR
448,bolgatanga,10.7856,-0.8514,13,81.25,0,4.45,GH
525,ovalle,-30.5983,-71.2003,39,80.94,0,9.84,CL


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = perfect_weather
hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,Humidity,Max Temperature,Cloudiness,Wind Speed,Country,Hotel Name
56,panaba,21.2833,-88.2667,44,87.96,0,10.85,MX,
98,coihaique,-45.5752,-72.0662,25,89.60,0,8.05,CL,
171,ancud,-41.8697,-73.8203,44,82.99,0,6.91,CL,
238,general roca,-39.0333,-67.5833,23,86.00,0,14.97,AR,
252,vallenar,-28.5708,-70.7581,29,83.86,0,9.98,CL,
323,bocana de paiwas,12.7857,-85.1227,55,83.88,3,9.66,NI,
407,rawson,-43.3002,-65.1023,27,82.44,0,29.89,AR,
448,bolgatanga,10.7856,-0.8514,13,81.25,0,4.45,GH,
525,ovalle,-30.5983,-71.2003,39,80.94,0,9.84,CL,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city = row["City"]
    
    params = {
    "location": (f"{latitude},{longitude}"),  
    "radius": 5000,
    "keyword": "hotel",
    "type": "lodging",
    "key": g_key
    }

    # Assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
   # Extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print(f"Hotel in {city} is {results[0]['name']}.")
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No result"''
        print("No results. Skipping...")
        
    print("------------------------------------------")
    

No results. Skipping...
------------------------------------------
Hotel in coihaique is Patagonia House.
------------------------------------------
Hotel in ancud is Panamericana Hotel Ancud.
------------------------------------------
Hotel in general roca is Estepa Hotel.
------------------------------------------
Hotel in vallenar is Hotel Del Marques.
------------------------------------------
No results. Skipping...
------------------------------------------
Hotel in rawson is Casa de mar.
------------------------------------------
Hotel in bolgatanga is TAP Hotel Ghana.
------------------------------------------
Hotel in ovalle is Hotel Limari.
------------------------------------------


In [12]:
hotel_df

,City,Latitude,Longitude,Humidity,Max Temperature,Cloudiness,Wind Speed,Country,Hotel Name
56,panaba,21.2833,-88.2667,44,87.96,0,10.85,MX,No result
98,coihaique,-45.5752,-72.0662,25,89.60,0,8.05,CL,Patagonia House
171,ancud,-41.8697,-73.8203,44,82.99,0,6.91,CL,Panamericana Hotel Ancud
238,general roca,-39.0333,-67.5833,23,86.00,0,14.97,AR,Estepa Hotel
252,vallenar,-28.5708,-70.7581,29,83.86,0,9.98,CL,Hotel Del Marques
323,bocana de paiwas,12.7857,-85.1227,55,83.88,3,9.66,NI,No result
407,rawson,-43.3002,-65.1023,27,82.44,0,29.89,AR,Casa de mar
448,bolgatanga,10.7856,-0.8514,13,81.25,0,4.45,GH,TAP Hotel Ghana
525,ovalle,-30.5983,-71.2003,39,80.94,0,9.84,CL,Hotel Limari


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))